In [83]:
import cv2
import sys
import numpy as np
import math

In [84]:
def my_skin_detect(src):
    dst = np.zeros(np.shape(src)[:-1], dtype=np.uint8)
    
    mask = np.logical_and.reduce((src[:,:,0] > 20, src[:,:,1] > 40, src[:,:,2] > 95, 
                                    src.max(axis=-1) - src.min(axis=-1) > 15, 
                                    abs(src[:,:,2] - src[:,:,1]) > 15, 
                                    src[:,:,2] > src[:,:,1], src[:,:,2] > src[:,:,0]))
    
    dst[mask] = 255
    
    return dst
    
def get_contours(cv_image_thres):
  contours, hierarchy = cv2.findContours(cv_image_thres, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  return contours

In [85]:
# ----------------
# a) Reading a stream of images from a webcamera, and displaying the video
# ----------------
# For more information on reading and writing video: http://docs.opencv.org/modules/highgui/doc/reading_and_writing_images_and_video.html
# open the video camera no. 0
cap = cv2.VideoCapture(0)

# if not successful, exit program
if not cap.isOpened():
    print("Cannot open the video cam")
    sys.exit()

# create a window called "MyVideo0"
cv2.namedWindow("MyVideo0", cv2.WINDOW_AUTOSIZE)

# read a new frame from video
ret, frame0 = cap.read()
if not ret:
    print("Cannot read a frame from video stream")

# show the frame in "MyVideo" window
cv2.imshow("MyVideo0", frame0)
cv2.namedWindow("rotate_hand", cv2.WINDOW_AUTOSIZE)

img_names = ['coyote.jpg', 'peace.jpg', 'fist.jpg','five.jpg']
img_temps = []
templateDict = dict()

for name in img_names:
    temp = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
    templateDict[name] = temp

"""
templateDict = dict()
for name in img_temps:
    temp = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
    w, h = temp.shape[::-1]
    tempList = []
    # tempList.append(temp)
    for i in range(6, 12, 2):
        resizedTemp = cv2.resize(temp, (int(w*float(i/10)), int(h*float(i/10))))
        tempList.append(resizedTemp)
    templateDict[name] = tempList.copy()
"""
    
method = eval('cv2.TM_SQDIFF_NORMED')

In [86]:
while(1):
    # read a new frame from video
    ret, frame = cap.read()
    # if not successful, break loop
    if not ret:
        print("Cannot read a frame from video stream")
        break

    cv2.imshow("MyVideo0", frame)

    # ----------------
    # b) Skin color detection
    # ----------------
    skin_mask = my_skin_detect(frame)
    
    skin_contours = get_contours(skin_mask)
    if(len(skin_contours) < 1):
        continue    
    maxContour = max(skin_contours, key=lambda x: cv2.contourArea(x))
    contour_mask = np.zeros(np.shape(frame)[:-1], dtype = np.uint8)
    cv2.fillPoly(contour_mask, [maxContour], (255))
    
    rect_frame = np.zeros_like(frame)
    
    x,y,w,h = cv2.boundingRect(maxContour)
    rectangle_mask = np.zeros(np.shape(frame)[:-1], dtype = np.uint8)
    
    cv2.rectangle(rectangle_mask,(x,y),(x+w,y+h),255,-1)
    
    rect_frame[rectangle_mask == 255] = frame[rectangle_mask == 255]
    
    M = cv2.moments(contour_mask, True)
    
    if(M["m00"] != 0):
        centroidX = int(M["m10"] / M["m00"])
        centroidY = int(M["m01"] / M["m00"])
        a = M["mu20"]
        b = 2.0 * M["mu11"]
        c = M["mu02"]
        if(a-c == 0):
            continue
        theta = math.atan(b/(a-c)) / 2.0
        theta = 180*theta / (math.pi)
        rotate_matrix = cv2.getRotationMatrix2D(center=(centroidX, centroidY), angle=theta, scale=1) 
        rotate_dest = cv2.warpAffine(frame, rotate_matrix, dsize = (frame.shape[:-1]))

    gray_rotate = cv2.cvtColor(rotate_dest, cv2.COLOR_BGR2GRAY)
    # cv2.rectangle(rotate_dest,(x-10,y-10),(x+w+10,y+h+10),255,3)

    maxVal = None
    maxLoc = -1
    maxKey = None

    dim = max(w,h)
    
    width, height = rotate_dest.shape[:-1]


    
    # if w > h:
    #     left_corner = (x-10, y + (h//2) - (w//2) - 10)
    # else:
    #     left_corner = (x + (w//2) - (h//2) - 10, y - 10)

    # cv2.rectangle(rotate_dest,left_corner,(left_corner[0]+dim+20, left_corner[1]+dim+20),(0,255,0),3)
    
    extra = int(28*dim/328.0)
    
    if w > h:
        left_corner = (x-extra//2, y + (h//2) - (w//2) - extra//2)
    else:
        left_corner = (x + (w//2) - (h//2) - extra//2, y - extra//2)

    cv2.rectangle(rotate_dest,left_corner,(left_corner[0]+dim+extra, left_corner[1]+dim+extra),(0,255,0),3)
    

    gray_rotate = cv2.cvtColor(rotate_dest, cv2.COLOR_BGR2GRAY)
    
    resize = gray_rotate[(left_corner[0]):(left_corner[0]+dim+extra+1), (left_corner[1]):(left_corner[1]+dim+extra+1)]
    
    if(min(resize.shape) < 2):
        continue
    resize_dest = cv2.resize(resize, (328,328))
    
    """
    template_resized = dict()
    for i in templateDict.keys():
        dim = max(w, h)
        template_resized[i] = cv2.resize(templateDict[i], (dim, dim))
    """

    for key in templateDict.keys():
        
        res = cv2.matchTemplate(image = resize_dest, templ = templateDict[key], method = method)
        minValTemp, maxValTemp, minLocTemp, maxLocTemp = cv2.minMaxLoc(res)
        if(maxVal is None or minValTemp < maxVal):
            maxVal = minValTemp
            maxLoc = minLocTemp
            maxKey = key
    
    top_left = maxLoc
        
    w = 300
    h = 300
        
    bottom_right = (maxLoc[0] + w, maxLoc[1] + h)

    cv2.rectangle(resize_dest, top_left, bottom_right, (0, 0, 255), 2)
    
    cv2.putText(resize_dest, str(maxKey), (top_left[0] +30, top_left[1] +30),cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 2)
    
    cv2.imshow("rotate_hand", rotate_dest)
    cv2.imshow("resize", resize_dest)
    
    if cv2.waitKey(30) == 27:
        print("esc key is pressed by user")
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Cannot read a frame from video stream


-1

In [82]:
import random
d = {"five":0, "fist":0, "coyote":0, "peace":0}
l = ["five", "fist", "coyote", "peace"]
order = []
for i in range(40):
    while True:
        rand = random.choice(l)
        if not order or (rand != order[-1] and d[rand] < 10):
            order.append(rand)
            d[rand] += 1
            break

print(order)
            


['peace', 'fist', 'coyote', 'five', 'peace', 'fist', 'five', 'peace', 'coyote', 'peace', 'five', 'coyote', 'peace', 'fist', 'peace', 'fist', 'peace', 'fist', 'five', 'fist', 'peace', 'fist', 'coyote', 'five', 'peace', 'five', 'fist', 'coyote', 'peace', 'five', 'coyote', 'five', 'coyote', 'five', 'fist', 'coyote', 'five', 'coyote', 'fist', 'coyote']
